#  Descripción del análisis de causas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
data= pd.read_csv('DATA_FINAL_08.csv',sep=';', header = 0)

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data['CARTERA'].value_counts()

0      3213264
-1      474001
0        21119
2        19132
1        10043
3         5100
-1        3162
4         1273
5          359
6          312
-2         282
2          177
1           75
-4          30
3           28
4           10
-9           8
-3           6
NG           3
5            2
-16          2
-36          1
-25          1
-12          1
-6           1
Name: CARTERA, dtype: int64

In [5]:
data['CARTERA']=np.where(data['CARTERA']=='0',0,data['CARTERA'])

In [6]:
data['INVOLUNTARIO']=np.where(data['CARTERA']==0,0,-1)
data['VOLUNTARIO']=np.where(data['CARTERA']==0,-1,0)

In [7]:
seleccionVariables=['CONGELACION','VIAJES','CAPACIDAD_DE_PAGO','CAUSAS_EXTERNAS','COMPETENCIA','MAL_SERVICIO','NO_UTILIZAR',
                    'NO_GUSTO','SIN_MOTIVO','SIN_CONTACTO','TRASLADOS','OTROS','CAMBIO_DE_SERVICIO','CAMBIO_TARIFA','EMPAQUETAMIENTO',
                    'CESION_DE_CONTRATO','UNIFICACION_EMPAQUETAMIENTO']

In [8]:
data['BAJAS_TOTAL']=data[seleccionVariables].sum(axis=1)

In [9]:
data.dtypes

CUENTA           int64
COD_SERVICIO    object
Velocidad       object
Internet        object
COD_NODO        object
                 ...  
AREA            object
ZONA            object
CIUDAD          object
DISTRITO        object
BAJAS_TOTAL      int64
Length: 88, dtype: object

In [10]:
data['CUENTA']=data['CUENTA'].astype(str)

In [11]:
maestro_data=data[['COD_NODO','DIVISION', 'AREA', 'ZONA', 'CIUDAD', 'DISTRITO']].drop_duplicates()

# SELECCION DE VARIABLES
* Alfa Tarifas
* Velocidad Beta
* Velocidad Factor Alfa
* Llamadas Admin
* Llamadas Tec
* Calidad muestras ok
* Razones

###  Descriptivo
* lugar Regional
* Churn Temprano
* Voluntario e involuntario

In [12]:
data['FECHA']

0           8
1           3
2           3
3           8
4           7
           ..
3748387    11
3748388    11
3748389     6
3748390     9
3748391    10
Name: FECHA, Length: 3748392, dtype: int64

### Calculo variables

In [13]:
### Alfa Beta

In [14]:
data[['ALFA','BETA']].describe()

,ALFA,BETA
count,3.162999e+06,3.507214e+06
mean,1.271766e+00,1.338653e+00
std,1.965219e+00,1.722602e+00
min,0.000000e+00,4.376013e-02
25%,4.642514e-01,8.435897e-01
50%,9.256882e-01,1.112853e+00
75%,1.648786e+00,1.485097e+00
max,3.215254e+02,1.840000e+02


In [15]:
#seleccion_variables=['COD_NODO','FECHA','INVOLUNTARIO','CD_BLANCOS_CAMBIOS_SERVICIO','AVG_TARIFA_TIGO','AVG_TARIFA_ETB','AVG_TARIFA_MOVISTAR','PROM_RENTA']
variables_tarifa=['AVG_TARIFA_TIGO','AVG_TARIFA_ETB','AVG_TARIFA_MOVISTAR']

In [16]:
data['comp_tarifa']=data[variables_tarifa].min(axis=1)

In [17]:
data['operador_menor_tarifa']=np.where(data['comp_tarifa']==data['AVG_TARIFA_TIGO'],'AVG_TARIFA_TIGO',
                                              np.where(data['comp_tarifa']==data['AVG_TARIFA_ETB'],'AVG_TARIFA_ETB',
                                                      np.where(data['comp_tarifa']==data['AVG_TARIFA_MOVISTAR'],'AVG_TARIFA_MOVISTAR','NINGUNO'
                                             )))

In [18]:
data['ALFA_tarifa']=np.where(data['operador_menor_tarifa']=='NINGUNO',np.nan,data['comp_tarifa']/data['PROM_RENTA'])

In [19]:
data[data['CUENTA']==2139171][['CUENTA','FECHA']+variables_tarifa+['PROM_RENTA','comp_tarifa','ALFA_tarifa']]

,CUENTA,FECHA,AVG_TARIFA_TIGO,AVG_TARIFA_ETB,AVG_TARIFA_MOVISTAR,PROM_RENTA,comp_tarifa,ALFA_tarifa


In [20]:
data[['PRECIO_MEGA_CLARO_1','PRECIO_MEGA_TIGO_1','PRECIO_MEGA_ETB_1']]

,PRECIO_MEGA_CLARO_1,PRECIO_MEGA_TIGO_1,PRECIO_MEGA_ETB_1
0,1044.000000,0.000000,0.000000
1,2071.826923,0.000000,0.000000
2,2043.260000,0.000000,0.000000
3,1311.897059,2933.333333,2663.333333
4,1260.892857,4666.666667,0.000000
...,...,...,...
3748387,1466.861538,0.000000,0.000000
3748388,1452.011494,0.000000,0.000000
3748389,2108.863636,3060.606061,2663.333333
3748390,1502.594203,1355.233333,0.000000


In [21]:
data.columns

Index(['CUENTA', 'COD_SERVICIO', 'Velocidad', 'Internet', 'COD_NODO',
       'ESTRATO', 'CARTERA', 'CONGELACION', 'VIAJES', 'CAPACIDAD_DE_PAGO',
       'CAUSAS_EXTERNAS', 'COMPETENCIA', 'MAL_SERVICIO', 'NO_UTILIZAR',
       'NO_GUSTO', 'SIN_MOTIVO', 'SIN_CONTACTO', 'TRASLADOS', 'OTROS',
       'CD_BLANCOS_CAMBIOS_OTROS', 'CAMBIO_DE_SERVICIO', 'CAMBIO_TARIFA',
       'EMPAQUETAMIENTO', 'CESION_DE_CONTRATO', 'UNIFICACION_EMPAQUETAMIENTO',
       'CD_BLANCOS_CAMBIOS_SERVICIO', 'FECHA', 'CHURN_TEMPRANO',
       'INVOLUNTARIO', 'VOLUNTARIO', 'TIPO_RAZON', 'CANTIDAD_LLAMADAS',
       'CANTIDAD_ADMIN', 'CANTIDAD_TECNICO', 'CAMPANA_HBO', 'CAMPANA_FOX',
       'CAMPANA_ADULTO', 'CAMPANA_EQUIPOS_ADIC', 'CAMPANA_ULTRAWIFI',
       'CAMPANA_VOZ', 'CAMPANA_RETENCION', 'CAMPANA_AFECTACION_SRV',
       'PART_CANT_CALL', 'CANT_USER', 'PROM_VEL', 'VEL_ME_8', 'VEL_10_25',
       'VEL_30', 'VEL_40_50', 'VEL_MA_60', 'CONVERGENTE', 'PROM_VEL_HOGAR',
       'MEDIANA_VEL', 'PROM_RENTA', 'PRECIO_MEGA_CLARO_1'

In [22]:
data['ALFA_tarifa'].describe()

count    2.192311e+06
mean     9.981074e-01
std      2.084416e-01
min      3.469364e-01
25%      8.491211e-01
50%      9.844099e-01
75%      1.129350e+00
max      2.918921e+00
Name: ALFA_tarifa, dtype: float64

### Llamadas

In [23]:
llamadas= pd.read_csv('Cantidad_llamadas_v3.csv',sep=';', header = 0)
#llamadas2= pd.read_csv('Cantidad_llamadas_v2.csv',sep=';', header = 0)

In [24]:
llamadas

,CUENTA,MES,CANTIDAD,ADMINISTRATIVO,TECNICO,NODO,ANIO
0,92508566,1,9,2,7,BT5,2021
1,76760738,1,1,1,0,S5D,2021
2,82231382,1,4,3,1,SUS,2021
3,41391822,1,6,4,2,1DN,2021
4,79054881,1,1,1,0,JGI2D,2021
...,...,...,...,...,...,...,...
9809865,93586402,12,1,1,0,2OE,2020
9809866,54213988,12,1,1,0,NGL,2020
9809867,74212184,12,1,0,1,TVZ,2020
9809868,48749462,12,1,0,1,POD,2020


In [25]:
#llamadas=llamadas.sort_values(['CUENTA','NODO','ANIO','MES'])

In [26]:
llamadas[['CUENTA','NODO','ANIO','MES']].groupby(['CUENTA','MES','ANIO']).count().sort_values('NODO', ascending=False)

NODO
CUENTA   MES ANIO      
61672518 3   2021     4
35208129 1   2021     4
28883684 6   2021     4
91882100 10  2021     3
39816957 10  2020     3
...                 ...
31356484 7   2021     1
         8   2021     1
         10  2020     1
             2021     1
99699954 10  2021     1

[9698560 rows x 1 columns]

In [27]:
llamadas[llamadas['CUENTA']==61672518].sort_values('MES')

,CUENTA,MES,CANTIDAD,ADMINISTRATIVO,TECNICO,NODO,ANIO
183849,61672518,3,2,2,0,2BQ2,2021
1144574,61672518,3,1,1,0,U1N,2021
3762790,61672518,3,3,3,0,1BQ12D,2021
4682471,61672518,3,4,2,2,1BQ13D,2021
2089553,61672518,4,1,1,0,1BQ12D,2021
5967842,61672518,8,2,2,0,1BQ12D,2021
8334485,61672518,11,2,1,0,1BQ13D,2020
8442124,61672518,12,2,1,0,1BQ13D,2020


In [28]:
llamadas['DIA']=1
llamadas['FECHA']=pd.to_datetime(llamadas['ANIO']*10000+llamadas['MES']*100+llamadas['DIA'], format='%Y%m%d')
llamadas['KEY']=llamadas['CUENTA'].astype(str)+'--'+llamadas['NODO'].astype(str)

In [29]:
pv_cantidad_llamadas=pd.pivot_table(llamadas, columns=['KEY'], values=['CANTIDAD'],index=['FECHA'], aggfunc=np.sum)
pv_cantidad_llamadas.columns= pv_cantidad_llamadas.columns.droplevel()
pv_cantidad_llamadas=pv_cantidad_llamadas.fillna(0).reset_index()
#pv_cantidad_llamadas

In [30]:
un_llamadas=pd.melt(pv_cantidad_llamadas, id_vars=['FECHA'], var_name='NODO', value_name='CANTIDAD_LLAMADAS')
un_llamadas=un_llamadas.sort_values(['NODO','FECHA'])
#un_llamadas

In [31]:
un_llamadas['CANT_1'] = un_llamadas.groupby(['NODO'])['CANTIDAD_LLAMADAS'].shift(1)
un_llamadas['CANT_2'] = un_llamadas.groupby(['NODO'])['CANTIDAD_LLAMADAS'].shift(2)
#un_llamadas

In [32]:
un_llamadas['Llamadas_3M']=un_llamadas[['CANTIDAD_LLAMADAS','CANT_1','CANT_2']].sum(axis=1)
un_llamadas[['CUENTA','COD_NODO']] = un_llamadas['NODO'].str.split('--',expand=True)

In [33]:
#un_llamadas=un_llamadas.rename(columns={'COD_NODO':'CUENTA','CUENTA':'COD_NODO'})
#un_llamadas['CUENTA']=un_llamadas['CUENTA'].astype('Int64')

In [34]:
len(un_llamadas)

43669632

In [35]:
un_llamadas=un_llamadas[un_llamadas['FECHA'].dt.year==2021]

In [36]:
un_llamadas['MES']=un_llamadas['FECHA'].dt.month

In [37]:
un_llamadas=un_llamadas.groupby(['CUENTA','COD_NODO','MES'])[['Llamadas_3M']].sum().reset_index()

In [38]:
un_llamadas

,CUENTA,COD_NODO,MES,Llamadas_3M
0,10000043,S1C,1,0.0
1,10000043,S1C,2,0.0
2,10000043,S1C,3,0.0
3,10000043,S1C,4,0.0
4,10000043,S1C,5,1.0
...,...,...,...,...
32752219,9999954,4A1012,8,17.0
32752220,9999954,4A1012,9,6.0
32752221,9999954,4A1012,10,3.0
32752222,9999954,4A1012,11,0.0


In [39]:
un_llamadas.dtypes['CUENTA']

dtype('O')

In [40]:
data=data.merge(un_llamadas[['COD_NODO','CUENTA','MES','Llamadas_3M']],left_on=['COD_NODO','CUENTA','FECHA'],right_on=['COD_NODO','CUENTA','MES'],how='left')

In [41]:
data[data['CUENTA']=='10000043']

,CUENTA,COD_SERVICIO,Velocidad,Internet,COD_NODO,ESTRATO,CARTERA,CONGELACION,VIAJES,CAPACIDAD_DE_PAGO,...,AREA,ZONA,CIUDAD,DISTRITO,BAJAS_TOTAL,comp_tarifa,operador_menor_tarifa,ALFA_tarifa,MES,Llamadas_3M
3133899,10000043,SUD,NaN,10,S1C,5,0,0,0,0,...,MEDELLIN,MEDELLIN OCCIDENTAL,MEDELLIN,MEDELLIN CENTRO OCCIDENTE,0,140000.0,AVG_TARIFA_TIGO,1.031262,3.0,0.0


#### Llamadas Admins

In [42]:
pv_cantidad_llamadas=pd.pivot_table(llamadas, columns=['KEY'], values=['ADMINISTRATIVO'],index=['FECHA'], aggfunc=np.sum)
pv_cantidad_llamadas.columns= pv_cantidad_llamadas.columns.droplevel()
pv_cantidad_llamadas=pv_cantidad_llamadas.fillna(0).reset_index()

In [43]:
un_llamadas=pd.melt(pv_cantidad_llamadas, id_vars=['FECHA'], var_name='NODO', value_name='ADMIN')
un_llamadas=un_llamadas.sort_values(['NODO','FECHA'])

In [44]:
un_llamadas['CANT_1'] = un_llamadas.groupby(['NODO'])['ADMIN'].shift(1)
un_llamadas['CANT_2'] = un_llamadas.groupby(['NODO'])['ADMIN'].shift(2)

In [45]:
un_llamadas['Llamadas_Admin_3M']=un_llamadas[['ADMIN','CANT_1','CANT_2']].sum(axis=1)
un_llamadas[['CUENTA','COD_NODO']] = un_llamadas['NODO'].str.split('--',expand=True)

In [46]:
un_llamadas=un_llamadas[un_llamadas['FECHA'].dt.year==2021]
un_llamadas['MES']=un_llamadas['FECHA'].dt.month

In [47]:
un_llamadas=un_llamadas.groupby(['CUENTA','COD_NODO','MES'])[['Llamadas_Admin_3M']].sum().reset_index()

In [48]:
data=data.merge(un_llamadas[['COD_NODO','CUENTA','MES','Llamadas_Admin_3M']],left_on=['COD_NODO','CUENTA','FECHA'],right_on=['COD_NODO','CUENTA','MES'],how='left')

In [49]:
data[data['CUENTA']=='10000043']

,CUENTA,COD_SERVICIO,Velocidad,Internet,COD_NODO,ESTRATO,CARTERA,CONGELACION,VIAJES,CAPACIDAD_DE_PAGO,...,CIUDAD,DISTRITO,BAJAS_TOTAL,comp_tarifa,operador_menor_tarifa,ALFA_tarifa,MES_x,Llamadas_3M,MES_y,Llamadas_Admin_3M
3133899,10000043,SUD,NaN,10,S1C,5,0,0,0,0,...,MEDELLIN,MEDELLIN CENTRO OCCIDENTE,0,140000.0,AVG_TARIFA_TIGO,1.031262,3.0,0.0,3.0,0.0


## Llamadas Técnicas

In [50]:
pv_cantidad_llamadas=pd.pivot_table(llamadas, columns=['KEY'], values=['TECNICO'],index=['FECHA'], aggfunc=np.sum)
pv_cantidad_llamadas.columns= pv_cantidad_llamadas.columns.droplevel()
pv_cantidad_llamadas=pv_cantidad_llamadas.fillna(0).reset_index()
un_llamadas=pd.melt(pv_cantidad_llamadas, id_vars=['FECHA'], var_name='NODO', value_name='TEC')
un_llamadas=un_llamadas.sort_values(['NODO','FECHA'])
un_llamadas['CANT_1'] = un_llamadas.groupby(['NODO'])['TEC'].shift(1)
un_llamadas['CANT_2'] = un_llamadas.groupby(['NODO'])['TEC'].shift(2)

In [51]:
un_llamadas['Llamadas_Tecn_3M']=un_llamadas[['TEC','CANT_1','CANT_2']].sum(axis=1)
un_llamadas[['CUENTA','COD_NODO']] = un_llamadas['NODO'].str.split('--',expand=True)
un_llamadas=un_llamadas[un_llamadas['FECHA'].dt.year==2021]
un_llamadas['MES']=un_llamadas['FECHA'].dt.month
un_llamadas=un_llamadas.groupby(['CUENTA','COD_NODO','MES'])[['Llamadas_Tecn_3M']].sum().reset_index()
data=data.merge(un_llamadas[['COD_NODO','CUENTA','MES','Llamadas_Tecn_3M']],left_on=['COD_NODO','CUENTA','FECHA'],right_on=['COD_NODO','CUENTA','MES'],how='left')

In [52]:
data[data['CUENTA']=='10000043']

,CUENTA,COD_SERVICIO,Velocidad,Internet,COD_NODO,ESTRATO,CARTERA,CONGELACION,VIAJES,CAPACIDAD_DE_PAGO,...,BAJAS_TOTAL,comp_tarifa,operador_menor_tarifa,ALFA_tarifa,MES_x,Llamadas_3M,MES_y,Llamadas_Admin_3M,MES,Llamadas_Tecn_3M
3133899,10000043,SUD,NaN,10,S1C,5,0,0,0,0,...,0,140000.0,AVG_TARIFA_TIGO,1.031262,3.0,0.0,3.0,0.0,3.0,0.0


In [53]:
data[['Llamadas_3M','Llamadas_Admin_3M','Llamadas_Tecn_3M']].describe()

,Llamadas_3M,Llamadas_Admin_3M,Llamadas_Tecn_3M
count,3.379624e+06,3.379624e+06,3.379624e+06
mean,2.895890e+00,1.888975e+00,9.412056e-01
std,3.931192e+00,2.801573e+00,1.894154e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,0.000000e+00
75%,4.000000e+00,3.000000e+00,1.000000e+00
max,3.730000e+02,2.500000e+02,2.380000e+02


In [54]:
data.drop(columns=['MES_x','MES_y','MES'],inplace=True)

In [55]:
###  VAlidar  ALFA y BETA (tarifa) por NODO

In [56]:
import gc
del un_llamadas
del pv_cantidad_llamadas
gc.collect()

123

### Calidad

In [57]:
#data_cal=pd.read_csv('user_rssi_device.csv',sep=';', header = 0)
#data_cal

In [58]:
#data_cal['MES']=(data_cal['fecha']%100)

In [59]:
#data.dtypes['FECHA']

In [60]:
#data_cal['MES']=data_cal['MES'].astype('Int64')

In [61]:
#data_cal['cuenta']=data_cal['cuenta'].astype('Int64')
#data_cal['cuenta']=data_cal['cuenta'].astype(str)

In [62]:
#data_cal=data_cal.groupby(['cuenta','MES'])['val_cantidad_muestras','val_muestras_ok'].sum().reset_index()

In [63]:
#data_cal

In [64]:
#data=data.merge(data_cal,left_on=['CUENTA','FECHA'],right_on=['cuenta','MES'],how='left')

In [65]:
#data['ind_calidad']=data['val_muestras_ok']/data['val_cantidad_muestras']

### Dataset Completo

In [66]:
data.columns

Index(['CUENTA', 'COD_SERVICIO', 'Velocidad', 'Internet', 'COD_NODO',
       'ESTRATO', 'CARTERA', 'CONGELACION', 'VIAJES', 'CAPACIDAD_DE_PAGO',
       'CAUSAS_EXTERNAS', 'COMPETENCIA', 'MAL_SERVICIO', 'NO_UTILIZAR',
       'NO_GUSTO', 'SIN_MOTIVO', 'SIN_CONTACTO', 'TRASLADOS', 'OTROS',
       'CD_BLANCOS_CAMBIOS_OTROS', 'CAMBIO_DE_SERVICIO', 'CAMBIO_TARIFA',
       'EMPAQUETAMIENTO', 'CESION_DE_CONTRATO', 'UNIFICACION_EMPAQUETAMIENTO',
       'CD_BLANCOS_CAMBIOS_SERVICIO', 'FECHA', 'CHURN_TEMPRANO',
       'INVOLUNTARIO', 'VOLUNTARIO', 'TIPO_RAZON', 'CANTIDAD_LLAMADAS',
       'CANTIDAD_ADMIN', 'CANTIDAD_TECNICO', 'CAMPANA_HBO', 'CAMPANA_FOX',
       'CAMPANA_ADULTO', 'CAMPANA_EQUIPOS_ADIC', 'CAMPANA_ULTRAWIFI',
       'CAMPANA_VOZ', 'CAMPANA_RETENCION', 'CAMPANA_AFECTACION_SRV',
       'PART_CANT_CALL', 'CANT_USER', 'PROM_VEL', 'VEL_ME_8', 'VEL_10_25',
       'VEL_30', 'VEL_40_50', 'VEL_MA_60', 'CONVERGENTE', 'PROM_VEL_HOGAR',
       'MEDIANA_VEL', 'PROM_RENTA', 'PRECIO_MEGA_CLARO_1'

In [67]:
sel_variables_descrip=['CUENTA','FECHA','COD_NODO','INVOLUNTARIO', 'VOLUNTARIO','DIVISION', 'AREA', 'ZONA', 'CIUDAD', 'DISTRITO','CHURN_TEMPRANO']
sel_variables_razones=['CONGELACION','VIAJES','CAPACIDAD_DE_PAGO','CAUSAS_EXTERNAS','COMPETENCIA','MAL_SERVICIO','NO_UTILIZAR',
                    'NO_GUSTO','SIN_MOTIVO','SIN_CONTACTO','TRASLADOS','OTROS','CAMBIO_DE_SERVICIO','CAMBIO_TARIFA','EMPAQUETAMIENTO',
                    'CESION_DE_CONTRATO','UNIFICACION_EMPAQUETAMIENTO','BAJAS_TOTAL']
sel_variables_causas2=['ALFA_tarifa','ALFA', 'BETA', 'MEGA','Llamadas_3M', 'Llamadas_Admin_3M', 'Llamadas_Tecn_3M','PROM_VEL_HOGAR']

In [68]:
data.to_csv('dataSetCompleto_actual.csv',index=False)

In [69]:
#data[sel_variables_descrip+sel_variables_razones+sel_variables_causas2].to_csv('dataSet_variables.csv')

## Procesar Nodos

In [70]:
data['VOLUNTARIO'].value_counts()

-1    3234383
 0     514009
Name: VOLUNTARIO, dtype: int64

In [71]:
sel_variables_descrip=['CUENTA','FECHA','COD_NODO','INVOLUNTARIO', 'VOLUNTARIO','DIVISION', 'AREA', 'ZONA', 'CIUDAD', 'DISTRITO','CHURN_TEMPRANO']
sel_variables_razones=['CONGELACION','VIAJES','CAPACIDAD_DE_PAGO','CAUSAS_EXTERNAS','COMPETENCIA','MAL_SERVICIO','NO_UTILIZAR',
                    'NO_GUSTO','SIN_MOTIVO','SIN_CONTACTO','TRASLADOS','OTROS','CAMBIO_DE_SERVICIO','CAMBIO_TARIFA','EMPAQUETAMIENTO',
                    'CESION_DE_CONTRATO','UNIFICACION_EMPAQUETAMIENTO','BAJAS_TOTAL']
sel_variables_causas2=['ALFA_tarifa','ALFA', 'BETA', 'MEGA','Llamadas_Admin_3M', 'Llamadas_Tecn_3M']

In [72]:
data=data[data['VOLUNTARIO']==-1]

In [73]:
for col in sel_variables_razones :
    data[col]=data[col]*-1

In [74]:
data=data[(data['BAJAS_TOTAL']>=1)&(data['BAJAS_TOTAL']<=2)]

In [75]:
data=data[(data['FECHA']>=1)&(data['FECHA']<=10)]

In [76]:
nodos_tab=data.groupby(['FECHA','COD_NODO'])[['CANT_USER']].min().reset_index()

In [77]:
nodos_tab_bajas=data.groupby(['FECHA','COD_NODO'])[sel_variables_razones].sum().reset_index()

In [78]:
data_comp=data[['COD_NODO','FECHA']+sel_variables_causas2].groupby(['COD_NODO','FECHA']).mean()
data_comp.reset_index(inplace=True)
data_comp.sort_values(['COD_NODO','FECHA'],inplace=True)

In [79]:
Nodos_Final=nodos_tab.merge(nodos_tab_bajas, left_on=['COD_NODO','FECHA'], right_on=['COD_NODO','FECHA'])
Nodos_Final=Nodos_Final.merge(data_comp, left_on=['COD_NODO','FECHA'], right_on=['COD_NODO','FECHA'])
Nodos_Final

,FECHA,COD_NODO,CANT_USER,CONGELACION,VIAJES,CAPACIDAD_DE_PAGO,CAUSAS_EXTERNAS,COMPETENCIA,MAL_SERVICIO,NO_UTILIZAR,...,EMPAQUETAMIENTO,CESION_DE_CONTRATO,UNIFICACION_EMPAQUETAMIENTO,BAJAS_TOTAL,ALFA_tarifa,ALFA,BETA,MEGA,Llamadas_Admin_3M,Llamadas_Tecn_3M
0,1,01S,159.0,0,0,0,0,0,1,0,...,0,0,0,1,NaN,0.420691,0.824528,0.510220,3.000,0.000000
1,1,02S,135.0,2,0,0,0,0,0,0,...,0,0,0,4,1.274914,0.757639,0.938875,0.806965,3.250,0.750000
2,1,03J,32.0,0,0,0,0,0,1,1,...,0,0,0,2,NaN,NaN,1.483871,NaN,6.000,0.500000
3,1,05J,63.0,0,0,1,0,0,0,0,...,0,0,0,3,NaN,NaN,NaN,3.477203,8.000,0.666667
4,1,0A6,96.0,1,0,0,0,0,0,0,...,0,0,0,2,NaN,0.063186,0.213823,0.295505,9.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106336,10,ZVG,299.0,1,0,0,0,0,0,0,...,0,0,0,2,1.041562,NaN,NaN,0.553299,3.000,0.500000
106337,10,ZVS,487.0,0,0,0,0,0,0,2,...,0,0,0,8,0.952015,6.214923,4.209790,1.476302,1.625,0.750000
106338,10,ZVT,141.0,1,0,0,0,0,0,0,...,0,0,0,2,0.542212,2.595187,1.575916,1.646780,2.000,0.000000
106339,10,ZVZ,445.0,0,0,0,0,0,0,1,...,0,0,0,5,NaN,0.583469,1.106707,0.527211,4.500,0.500000


In [80]:
Nodos_Final['Llamadas_Admin_3M']=Nodos_Final['Llamadas_Admin_3M']/Nodos_Final['CANT_USER']
Nodos_Final['Llamadas_Tecn_3M']=Nodos_Final['Llamadas_Tecn_3M']/Nodos_Final['CANT_USER']

In [81]:
Nodos_Final['ind_churn']=np.where(Nodos_Final['CANT_USER']>0,Nodos_Final['BAJAS_TOTAL']/Nodos_Final['CANT_USER'],np.nan)

## Seleccion de Nodos

In [82]:
print('Nodos: ',Nodos_Final['COD_NODO'].nunique(), ' Registros (nodosxmes): ', len(Nodos_Final))

Nodos:  14065  Registros (nodosxmes):  106341


In [83]:
Nodos_Final_fil=Nodos_Final[Nodos_Final['CANT_USER']>30]

In [84]:
print('Nodos: ',Nodos_Final_fil['COD_NODO'].nunique(), ' Registros (nodosxmes): ', len(Nodos_Final_fil))

Nodos:  12624  Registros (nodosxmes):  103026


In [85]:
Nodos_Final_fil=Nodos_Final_fil.merge(Nodos_Final_fil['COD_NODO'].value_counts().to_frame(), left_on='COD_NODO', right_index=True, how='left')

In [86]:
Nodos_Final_fil=Nodos_Final_fil[Nodos_Final_fil['COD_NODO_y']==10]

In [87]:
Nodos_Final_fil.rename(columns={'COD_NODO_x':'COD_NODO','COD_NODO_y':'nMeses'},inplace=True)

In [88]:
print('Nodos: ',Nodos_Final_fil['COD_NODO'].nunique(), ' Registros (nodosxmes): ', len(Nodos_Final_fil))

Nodos:  5481  Registros (nodosxmes):  54810


In [89]:
data_comp=Nodos_Final_fil[['COD_NODO','FECHA']+sel_variables_causas2].groupby(['COD_NODO','FECHA']).mean()
data_comp.reset_index(inplace=True)
data_comp.sort_values(['COD_NODO','FECHA'],inplace=True)
data_comp_pt=pd.pivot_table(data_comp,index='FECHA',columns='COD_NODO')

In [90]:
list_eliminar=[]
i=0
for col in sel_variables_causas2:
    print(col)
    df_aux=data_comp_pt[col].isnull().sum().to_frame()
    df_aux=df_aux[df_aux[0]>2]
    if i==0:
        list_eliminar=list(df_aux.index)
    else :
        list_eliminar=list_eliminar+list(df_aux.index)
    i=i+1
list_eliminar = list(set(list_eliminar))

ALFA_tarifa
ALFA
BETA
MEGA
Llamadas_Admin_3M
Llamadas_Tecn_3M


In [91]:
len(list_eliminar)

846

In [92]:
data_comp_pt=data_comp_pt.interpolate()
data_comp_pt.reset_index(inplace=True)
data_comp_un=data_comp_pt.melt(id_vars=['FECHA'], var_name=['varible','COD_NODO'],value_name='valor')
data_comp=pd.pivot_table(data_comp_un,index=['COD_NODO','FECHA'],columns='varible')
data_comp.columns = data_comp.columns.droplevel()
data_comp.reset_index(inplace=True)

In [93]:
Nodos_Final_fil=Nodos_Final_fil.drop(columns=sel_variables_causas2).merge(data_comp,left_on=['COD_NODO','FECHA'],right_on=['COD_NODO','FECHA'])

In [94]:
### Eliminar Nodos con mayor imputaciones
Nodos_Final_fil=Nodos_Final_fil[~Nodos_Final_fil['COD_NODO'].isin(list_eliminar)]

In [95]:
print('Nodos: ',Nodos_Final_fil['COD_NODO'].nunique(), ' Registros (nodosxmes): ', len(Nodos_Final_fil))

Nodos:  4635  Registros (nodosxmes):  46349


In [96]:
Nodos_Final_fil=Nodos_Final_fil.merge(maestro_data, left_on=['COD_NODO'], right_on=['COD_NODO'], how='left')

In [97]:
print('Nodos: ',Nodos_Final_fil['COD_NODO'].nunique(), ' Registros (nodosxmes): ', len(Nodos_Final_fil))

Nodos:  4635  Registros (nodosxmes):  46349


In [98]:
Nodos_Final_fil.to_csv('NODOS_FILTRADOS_base1.csv',sep=';',index=False)

###  Descripción del dataset generado

In [99]:
Nodos_Final_fil.columns

Index(['FECHA', 'COD_NODO', 'CANT_USER', 'CONGELACION', 'VIAJES',
       'CAPACIDAD_DE_PAGO', 'CAUSAS_EXTERNAS', 'COMPETENCIA', 'MAL_SERVICIO',
       'NO_UTILIZAR', 'NO_GUSTO', 'SIN_MOTIVO', 'SIN_CONTACTO', 'TRASLADOS',
       'OTROS', 'CAMBIO_DE_SERVICIO', 'CAMBIO_TARIFA', 'EMPAQUETAMIENTO',
       'CESION_DE_CONTRATO', 'UNIFICACION_EMPAQUETAMIENTO', 'BAJAS_TOTAL',
       'ind_churn', 'nMeses', 'ALFA', 'ALFA_tarifa', 'BETA',
       'Llamadas_Admin_3M', 'Llamadas_Tecn_3M', 'MEGA', 'DIVISION', 'AREA',
       'ZONA', 'CIUDAD', 'DISTRITO'],
      dtype='object')